# Análise de sub-regiões

In [1]:
import math
las_vegas_lat=36.114647
las_vegas_long=-115.172813
las_vegas_area=351.90 # km^2
lat_degree_to_km=110.567# km
long_degree_to_km=111.699# km

In [2]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    # deg to rad
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2-lon1
    dlat = lat2-lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2*asin(sqrt(a))
    r = 6371 # km
    return c*r
def km_to_deg(value):
    return value/haversine(0,0,0,1)

In [39]:
haversine(0,0,0,1)

111.19492664455873

In [3]:
las_vegas_lat

36.114647

In [4]:
las_vegas_lat+km_to_deg(math.sqrt(las_vegas_area))

36.28335072063827

In [5]:
las_vegas_lat-km_to_deg(math.sqrt(las_vegas_area))

35.94594327936173

In [6]:
haversine(0,36.389326,0,36.123935)

29.51013277712499

In [7]:
haversine(-115.427600,0,-115.048827,0)

42.11763594993774

In [8]:
print(_,"*",__,"=",_*__)


42.11763594993774 * 29.51013277712499 = 1242.8970291412757


In [10]:
# lat degree
# long degree
# area km^2
def define_area(lat,long,area):
    distance_from_origin=math.sqrt(area)/2
    degree_from_origin=km_to_deg(distance_from_origin)
    return {"long":[long-degree_from_origin,long+degree_from_origin],
            "lat":[lat-degree_from_origin,lat+degree_from_origin]}
define_area(las_vegas_lat,las_vegas_long,las_vegas_area)

{'long': [-115.25716486031914, -115.08846113968087],
 'lat': [36.03029513968087, 36.19899886031913]}

Resultados imprecisos

usar a padrão

# Algoritmos sobre área e subárea

In [33]:
def delimiter_area(case):
    if case == 1:
        print('Area selected: Las Vegas')
        city = "LasVegas"
        area = {}
        area['city'] = city.lower()
        area['final_latitude'] = 36.389326
        area['initial_latitude'] = 36.123935
        area['initial_longitude'] = -115.427600
        area['final_longitude'] = -115.048827
    elif case == 2: 
        print('Area selected: Phoenix')
        city = "Phoenix"
        area = {}
        area['city'] = city.lower()
        area['final_latitude'] = 34.003012
        area['initial_latitude'] = 33.006796
        area['initial_longitude'] = -112.606674
        area['final_longitude'] = -111.381699
    #34.995653, -81.034521 35.400766, -80.651372
    elif case == 3: 
        print('Area selected: Charlotte')
        city = "Charlotte"
        area = {}
        area['city'] = city.lower()
        area['final_latitude'] = 35.400766
        area['initial_latitude'] = 34.995653
        area['initial_longitude'] = -81.034521
        area['final_longitude'] = -80.651372
    elif case == 4:
        print('Area selected: Madison')
        city = "Madison"
        area = {}
        area['city'] = city.lower()
        area['final_latitude'] = 43.215156
        area['initial_latitude'] = 42.936791
        area['initial_longitude'] = -89.608990
        area['final_longitude'] = -89.179837
    return area

area=delimiter_area(1)

Area selected: Las Vegas


In [35]:
import pandas as pd

df_business=pd.read_csv("../data/business.csv",usecols=['business_id','latitude','longitude','city'])

In [37]:
def pois_in_area(area,df_business):
    return df_business[(df_business['latitude']>=area['initial_latitude']) &\
                (df_business['latitude']<=area['final_latitude'])&\
                (df_business['longitude']>=area['initial_longitude']) &\
                (df_business['longitude']<=area['final_longitude'])]

df_business_in_area=pois_in_area(area,df_business)
df_business=None
df_business_in_area.head()

,business_id,city,latitude,longitude
20,dFMxzHygTy6F873843dHAA,Las Vegas,36.169993,-115.140685
21,lxnuq9wJiwLOPJ4uZU2ljg,North Las Vegas,36.225852,-115.132800
24,BsMdebN4nZySpGTfXr-7yg,Las Vegas,36.271546,-115.264484
28,iojTeSaoPuxm4WeCzDUA6w,Las Vegas,36.129424,-115.184443
30,LB6ZyCfUzeX9OLdunHhnOQ,Las Vegas,36.161805,-115.287277


In [38]:
del df_business_in_area['city']

In [80]:
def poi_set_subarea(area,df_business_in_area,distance_km):
    '''
        area: is for example a city like phoenix, las vegas, new york, etc.
        df_business_in_area: is a dataframe of business filtered in a area
        distance_km: is the distance of subareas or area in km^2
    '''
    longitude_delta = abs(area['final_longitude']-area['initial_longitude'])
    latitude_delta = abs(area['final_latitude']-area['initial_latitude'])
    avg_latitude = (area['initial_latitude']+area['final_latitude'])/2.0
    LAT_DEGREE_KM_EQUATOR=111.0 # 110.57
    LONG_DEGREE_KM_EQUATOR=111.321 # 111.32
    # define step degree in latitude
    subarea_latitude_delta_degree = distance_km/LAT_DEGREE_KM_EQUATOR
    # define step degree in longitude
    subarea_longitude_delta_degree = distance_km/(LONG_DEGREE_KM_EQUATOR * math.cos(avg_latitude * math.pi/180))
    # number of subareas
    num_subareas = math.ceil(longitude_delta/subarea_longitude_delta_degree) * math.ceil(latitude_delta/subarea_latitude_delta_degree)
    
    df_business_in_area['subarea_id']=\
    np.abs(np.ceil((df_business_in_area['longitude']-area['final_longitude'])/subarea_longitude_delta_degree))+\
    (np.abs(np.ceil((df_business_in_area['latitude']-area['initial_latitude'])/subarea_latitude_delta_degree))-1)\
    * (np.ceil(longitude_delta/subarea_longitude_delta_degree))
#    Code that explains the above logic 
#
#     for index,row in df_business_in_area.iterrows():
#         latitude_poi_in_subarea = (row['latitude']-area['initial_latitude'])/subarea_latitude_delta_degree
#         longitude_poi_in_subarea = (row['longitude']-area['final_longitude'])/subarea_longitude_delta_degree
#         line = abs(math.ceil(latitude_poi_in_subarea))            
#         column = abs(math.ceil(longitude_poi_in_subarea))
#         subarea_id = column + (line -1) * (math.ceil(longitude_delta/subarea_longitude_delta_degree))
#         row['subarea_id']=subarea_id
    return df_business_in_area

poi_set_subarea(area,df_business_in_area,0.5).head()


,business_id,latitude,longitude,subarea_id
20,dFMxzHygTy6F873843dHAA,36.169993,-115.140685,706.0
21,lxnuq9wJiwLOPJ4uZU2ljg,36.225852,-115.132800,1533.0
24,BsMdebN4nZySpGTfXr-7yg,36.271546,-115.264484,2246.0
28,iojTeSaoPuxm4WeCzDUA6w,36.129424,-115.184443,93.0
30,LB6ZyCfUzeX9OLdunHhnOQ,36.161805,-115.287277,594.0
